## Business Problem

Small Business Administration (SBA) adalah sebuah lembaga independen pemerintahan Amerika yang membantu para pengusaha di berbagai macam industri dalam bentuk jaminan dari berbagai lembaga keuangan seperti bank.

Di A.S. pengusaha bisnis kecil sebenarnya tidak meminjam dari pemerintah, mereka masih meminjam dari bank, tapi SBA akan menjamin sebagian dari pinjaman mereka, sehingga memudahkan untuk memenuhi syarat dan untuk mencapai tingkat bunga yang lebih rendah. Ini adalah cara pemerintah untuk mendorong bank memberikan pinjaman kepada usaha kecil.

Jika pengusaha bisnis kecil memenuhi persyaratan untuk program pinjaman usaha kecil umum SBA. Mereka harus memenuhi persyaratan pemerintah yang terkadang kompleks dan mengumpulkan aplikasi yang komprehensif, dan kemudian menunggu sampai diproses, sehingga butuh beberapa waktu tambahan.

Pada project ini, saya akan coba memprediksi kemungkinan peminjam/pengusaha ini gagal bayar atau tidak berdasarkan data yang telah dikumpulkan dari tahun 1987–2014 menggunakan machine learning sehingga dapat menghemat waktu dan memudahkan untuk memutuskan untuk beri pinjaman atau tidak ke usaha kecil yang segera membutuhkan pinjaman.

1. LoanNr_ChkDgt : Identifier – Primary key
2. Name : Borrower name
3. City : Borrower city
4. State : Borrower state
5. Zip : Borrower zip code
6. Bank : Bank name
7. BankState : Bank state
8. NAICS : North American industry classification system code
9. ApprovalDate : Date SBA commitment issued
10. ApprovalFY : Fiscal year of commitment
11. Term : Loan term in months
12. NoEmp : Number of business employees
13. NewExist : 1 = Existing business, 2 = New business
14. CreateJob : Number of jobs created
15. RetainedJob : Number of jobs retained
16. FranchiseCode : Franchise code, (00000 or 00001) = No franchise
17. UrbanRural : 1 = Urban, 2 = rural, 0 = undefined
18. RevLineCr : Revolving line of credit: Y = Yes, N = No
19. LowDoc : LowDoc Loan Program: Y = Yes, N = No
20. ChgOffDate : The date when a loan is declared to be in default
21. DisbursementDate : Disbursement date
22. DisbursementGross : Amount disbursed
23. BalanceGross : Gross amount outstanding
24. MIS_Status : Loan status charged off = CHGOFF, Paid in full = PIF
25. ChgOffPrinGr : Charged-off amount
26. GrAppv : Gross amount of loan approved by bank
27. SBA_Appv : SBA’s guaranteed amount of approved loan

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
SBA = pd.read_csv('SBAnational.csv')
SBA.head()

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [3]:
SBA.describe()

,LoanNr_ChkDgt,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural
count,8.991640e+05,899164.000000,899164.000000,899164.000000,899164.000000,899028.000000,899164.000000,899164.000000,899164.000000,899164.000000
mean,4.772612e+09,53804.391241,398660.950146,110.773078,11.411353,1.280404,8.430376,10.797257,2753.725933,0.757748
std,2.538175e+09,31184.159152,263318.312759,78.857305,74.108196,0.451750,236.688165,237.120600,12758.019136,0.646436
min,1.000014e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.589758e+09,27587.000000,235210.000000,60.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.361439e+09,55410.000000,445310.000000,84.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,6.904627e+09,83704.000000,561730.000000,120.000000,10.000000,2.000000,1.000000,4.000000,1.000000,1.000000
max,9.996003e+09,99999.000000,928120.000000,569.000000,9999.000000,2.000000,8800.000000,9500.000000,99999.000000,2.000000


In [4]:
SBA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [5]:
SBA.isna().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

In [6]:
for i in SBA :
    print(i)
    print(SBA[i].nunique())
    print("\r")

LoanNr_ChkDgt
899164

Name
779583

City
32581

State
51

Zip
33611

Bank
5802

BankState
56

NAICS
1312

ApprovalDate
9859

ApprovalFY
70

Term
412

NoEmp
599

NewExist
3

CreateJob
246

RetainedJob
358

FranchiseCode
2768

UrbanRural
3

RevLineCr
18

LowDoc
8

ChgOffDate
6448

DisbursementDate
8472

DisbursementGross
118859

BalanceGross
15

MIS_Status
2

ChgOffPrinGr
83165

GrAppv
22128

SBA_Appv
38326



## Preprocessing

### Date (ApprovalDate, ChgOffDate, DisbursementDate)

In [7]:
SBA[['ApprovalDate', 'ChgOffDate', 'DisbursementDate']].head()

,ApprovalDate,ChgOffDate,DisbursementDate
0,28-Feb-97,NaN,28-Feb-99
1,28-Feb-97,NaN,31-May-97
2,28-Feb-97,NaN,31-Dec-97
3,28-Feb-97,NaN,30-Jun-97
4,28-Feb-97,NaN,14-May-97


In [8]:
SBA['ChgOffDate'].unique()

array([nan, '24-Jun-91', '18-Apr-02', ..., '25-Dec-02', '11-Jul-00',
       '9-Oct-98'], dtype=object)

In [9]:
SBA['ApprovalDate'] = pd.to_datetime(SBA['ApprovalDate'],format='%d-%b-%y')

In [10]:
SBA['ChgOffDate'] = pd.to_datetime(SBA['ChgOffDate'],format='%d-%b-%y')

In [11]:
SBA['DisbursementDate'] = pd.to_datetime(SBA['DisbursementDate'],format='%d-%b-%y')

In [12]:
SBA[['ApprovalDate', 'ChgOffDate', 'DisbursementDate']].head()

,ApprovalDate,ChgOffDate,DisbursementDate
0,1997-02-28,NaT,1999-02-28
1,1997-02-28,NaT,1997-05-31
2,1997-02-28,NaT,1997-12-31
3,1997-02-28,NaT,1997-06-30
4,1997-02-28,NaT,1997-05-14


In [13]:
SBA['ChgOffDate'].unique()

array([                          'NaT', '1991-06-24T00:00:00.000000000',
       '2002-04-18T00:00:00.000000000', ...,
       '2002-12-25T00:00:00.000000000', '2000-07-11T00:00:00.000000000',
       '1998-10-09T00:00:00.000000000'], dtype='datetime64[ns]')

## ApprovalFY

In [14]:
SBA['ApprovalFY'].unique()

array([1997, 1980, 2006, 1998, 1999, 2000, 2001, 1972, 2003, 2004, 1978,
       1979, 1981, 2005, 1962, 1982, 1965, 1966, 1983, 1973, 1984, 2007,
       1985, 1986, 1987, 2008, 1988, 2009, 1989, 1991, 1990, 1974, 2010,
       1967, 2011, 1992, 1993, 2002, 2012, 2013, 1994, 2014, 1975, 1977,
       1976, 1968, '2004', '1994', '1979', '1976', '1967', '1975', '1974',
       '1977', '1981', '1982', '1983', '1984', '1978', '1980', '1968',
       '1976A', '1969', '1995', '1970', '2005', '1996', '1971', 1996,
       1971], dtype=object)

In [15]:
# Setelah di cek, ada 1976A, akan diubah menjadi tahun 1976 

SBA['ApprovalFY'].replace('1976A', 1976, inplace = True)
SBA['ApprovalFY'].unique()

array([1997, 1980, 2006, 1998, 1999, 2000, 2001, 1972, 2003, 2004, 1978,
       1979, 1981, 2005, 1962, 1982, 1965, 1966, 1983, 1973, 1984, 2007,
       1985, 1986, 1987, 2008, 1988, 2009, 1989, 1991, 1990, 1974, 2010,
       1967, 2011, 1992, 1993, 2002, 2012, 2013, 1994, 2014, 1975, 1977,
       1976, 1968, '2004', '1994', '1979', '1976', '1967', '1975', '1974',
       '1977', '1981', '1982', '1983', '1984', '1978', '1980', '1968',
       '1969', '1995', '1970', '2005', '1996', '1971', 1996, 1971],
      dtype=object)

In [16]:
# approvalFY akan diubah menjadi integer
SBA['ApprovalFY']= SBA['ApprovalFY'].astype(int)

In [17]:
SBA['ApprovalFY'].unique()

array([1997, 1980, 2006, 1998, 1999, 2000, 2001, 1972, 2003, 2004, 1978,
       1979, 1981, 2005, 1962, 1982, 1965, 1966, 1983, 1973, 1984, 2007,
       1985, 1986, 1987, 2008, 1988, 2009, 1989, 1991, 1990, 1974, 2010,
       1967, 2011, 1992, 1993, 2002, 2012, 2013, 1994, 2014, 1975, 1977,
       1976, 1968, 1969, 1995, 1970, 1996, 1971])

## Currency (DisbursementGross, BalanceGross, ChgOffPrinGr, GrAppv, SBA_Appv)

In [18]:
# Kolom Currency akan diubah menjadi float
curr_col = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
SBA[curr_col].head()

,DisbursementGross,BalanceGross,ChgOffPrinGr,GrAppv,SBA_Appv
0,"$60,000.00",$0.00,$0.00,"$60,000.00","$48,000.00"
1,"$40,000.00",$0.00,$0.00,"$40,000.00","$32,000.00"
2,"$287,000.00",$0.00,$0.00,"$287,000.00","$215,250.00"
3,"$35,000.00",$0.00,$0.00,"$35,000.00","$28,000.00"
4,"$229,000.00",$0.00,$0.00,"$229,000.00","$229,000.00"


In [19]:
SBA['DisbursementGross'].dtypes

dtype('O')

In [20]:
SBA[curr_col[0]]

0          $60,000.00 
1          $40,000.00 
2         $287,000.00 
3          $35,000.00 
4         $229,000.00 
              ...     
899159     $70,000.00 
899160     $85,000.00 
899161    $300,000.00 
899162     $75,000.00 
899163     $30,000.00 
Name: DisbursementGross, Length: 899164, dtype: object

In [21]:
# Merubah kolom currency menjadi float

def remove_dollar(x) :
    k = x.replace('$', '')
    l = k.replace(' ', '')
    m = l.replace(',', '')
    return float(m)

for i in range(5) :
    SBA[curr_col[i]] = SBA[curr_col[i]].apply(lambda x : remove_dollar(x))
    
SBA[curr_col].head()

,DisbursementGross,BalanceGross,ChgOffPrinGr,GrAppv,SBA_Appv
0,60000.0,0.0,0.0,60000.0,48000.0
1,40000.0,0.0,0.0,40000.0,32000.0
2,287000.0,0.0,0.0,287000.0,215250.0
3,35000.0,0.0,0.0,35000.0,28000.0
4,229000.0,0.0,0.0,229000.0,229000.0


## MIS_Status

In [22]:
SBA['MIS_Status'].unique()

array(['P I F', 'CHGOFF', nan], dtype=object)

In [23]:
SBA['MIS_Status'].value_counts()

P I F     739609
CHGOFF    157558
Name: MIS_Status, dtype: int64

In [24]:
SBA['MIS_Status'].isna().sum()

1997

In [25]:
# Merubah MIS_Status dari string menjadi integer
SBA['MIS_Status'] = SBA['MIS_Status'].replace({'P I F' : 0, 'CHGOFF' : 1})
SBA.MIS_Status.value_counts()

0.0    739609
1.0    157558
Name: MIS_Status, dtype: int64

In [26]:
SBA['MIS_Status'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: MIS_Status, dtype: float64

Ingat, MIS_Status (status pinjaman ada 2 jenis), dibayar penuh dan pinjaman dibebankan (tidak dibayar penuh)

ChgOffDate adalah tanggal saat pinjaman gagal bayar. Jadi, kalau ada tanggalnya sudah pasti status pinjaman dibebankan. Kalau tidak ada tanggal di ChgOffDate berarti status pinjaman dibayar penuh.

In [27]:
# Mengisi input yang kosong pada MIS_Status dengan CghOffDate dengan mencocokan indexnya

SBA['MIS_Status'] = np.where((SBA['MIS_Status'] == 0.0) & (SBA['ChgOffDate'] == np.nan),0, SBA.MIS_Status)
SBA['MIS_Status'] = np.where((SBA['MIS_Status'] == 1.0) & (SBA['ChgOffDate'] != np.nan),1, SBA.MIS_Status)

SBA = SBA[(SBA['MIS_Status'] == 0) | (SBA['MIS_Status'] == 1)]

In [28]:
print(SBA[['MIS_Status', 'ChgOffDate']].head(10))

   MIS_Status ChgOffDate
0         0.0        NaT
1         0.0        NaT
2         0.0        NaT
3         0.0        NaT
4         0.0        NaT
5         0.0        NaT
6         1.0 1991-06-24
7         0.0        NaT
8         0.0        NaT
9         0.0        NaT


In [29]:
SBA.MIS_Status.value_counts()

0.0    739609
1.0    157558
Name: MIS_Status, dtype: int64

In [30]:
SBA['MIS_Status'].isna().sum()

0

## LowDoc

In [31]:
SBA['LowDoc'].head()

0    Y
1    Y
2    N
3    Y
4    N
Name: LowDoc, dtype: object

In [32]:
SBA['LowDoc'].unique()

array(['Y', 'N', 'C', '1', nan, 'S', 'R', 'A', '0'], dtype=object)

In [33]:
SBA['LowDoc'].isna().sum()

2578

In [34]:
SBA['LowDoc'].value_counts()

N    780997
Y    110171
0      1490
C       758
S       603
A       495
R        74
1         1
Name: LowDoc, dtype: int64

In [35]:
((SBA['LowDoc'] == '0') | (SBA['LowDoc'] == 'C') | (SBA['LowDoc'] == 'S') | (SBA['LowDoc'] == 'A') | (SBA['LowDoc'] == 'R') | (SBA['LowDoc'] == '1')).value_counts()

False    893746
True       3421
Name: LowDoc, dtype: int64

In [36]:
# Selain 'Y' dan 'N' akan diubah menjadi nan, akan dihitung berapa jumlah nan yang ada
SBA['LowDoc'] = SBA['LowDoc'].replace({'C': np.nan, 'S': np.nan, 'A': np.nan, 'R': np.nan, '1': np.nan, '0' : np.nan})

In [37]:
SBA['LowDoc'].unique()

array(['Y', 'N', nan], dtype=object)

In [38]:
#tadinya mau coba pake ini tapi menurut saya lebih mudah langsung di replace

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

LowDoc = pd.DataFrame(SBA.LowDoc)
LowDoc.head()

transformer = ColumnTransformer([
    ('encoder', OneHotEncoder(), ['LowDoc'])
])

LowDoc_encoded = pd.DataFrame(transformer.fit_transform(LowDoc))
LowDoc_encoded = LowDoc_encoded.astype(int)
LowDoc_encoded.head()

,0,1,2
0,0,1,0
1,0,1,0
2,1,0,0
3,0,1,0
4,1,0,0


In [39]:
SBA['LowDoc'].value_counts()

N    780997
Y    110171
Name: LowDoc, dtype: int64

In [40]:
SBA['LowDoc'].isna().sum()

5999

In [41]:
(SBA['LowDoc'].isna().sum() / (len(SBA['LowDoc'])))*100

0.6686603497453651

Ada 0.6% baris yang kosong.

Berdasarkan jurnal Should This Loan be Approved or Denied?”: A Large Dataset with Class Assignment Guidelines
Jika pinjaman <150000 maka 'Yes' (dapat diproses dengan 1 halaman dokumen aplikasi)
Jika pinjaman <150000 maka 'No' (tidak dapat diproses dengan 1 halaman dokumen aplikasi)

Sehingga akan diisi kolom LowDoc menggunakan kolom DisbursementGross

In [42]:
SBA['LowDoc'] = np.where((SBA['LowDoc'] == np.nan) & (SBA['DisbursementGross'] < 150000), 'Y', SBA.LowDoc)
SBA['LowDoc'] = np.where((SBA['LowDoc'] == np.nan) & (SBA['DisbursementGross'] >= 150000), 'N', SBA.LowDoc)

SBA.LowDoc.value_counts()

N    780997
Y    110171
Name: LowDoc, dtype: int64

In [43]:
SBA['LowDoc'] = SBA['LowDoc'].replace({'N' : 0, 'Y' : 1})

In [44]:
SBA.LowDoc.value_counts()

0.0    780997
1.0    110171
Name: LowDoc, dtype: int64

In [45]:
SBA.LowDoc.isna().sum()

5999

## RevLineCr

In [46]:
SBA['RevLineCr'].unique()

array(['N', '0', 'Y', 'T', nan, '`', ',', '1', 'C', '3', '2', 'R', '7',
       'A', '5', '.', '4', '-', 'Q'], dtype=object)

In [47]:
# merubah semua baris selain 'Y' dan 'N' menjadi Nan

SBA['RevLineCr'] = np.where((SBA['RevLineCr'] != 'Y') & (SBA['RevLineCr'] != 'N'), np.nan, SBA.RevLineCr)

In [48]:
SBA['RevLineCr'].unique()

array(['N', nan, 'Y'], dtype=object)

In [49]:
# Merubah kolom 'RevLineCr' menjadi integer

SBA['RevLineCr'] = SBA['RevLineCr'].replace({'N' : 0, 'Y' : 1})

In [50]:
SBA.RevLineCr.value_counts()

0.0    419252
1.0    200660
Name: RevLineCr, dtype: int64

In [51]:
SBA.RevLineCr.isna().sum()

277255

## NewExist

In [52]:
SBA['NewExist'].unique()

array([ 2.,  1.,  0., nan])

In [53]:
SBA['NewExist'].dtype

dtype('float64')

In [54]:
# Akan dirubah 
# 1. = Existing business menjadi 0 (integer),
# 2. = New business menjadi 1 (integer), dan
# 0. dan nan menjadi 2 (tujuannya untuk mengubah menjadi nan tapi saya gunakan angka terlebih dahulu agar memudahkan merubah menjadi int)

SBA['NewExist'] = SBA['NewExist'].replace({1.0 : 0, 2.0 : 1, np.nan : 2, 0. : 2 }).astype(int)

In [55]:
SBA['NewExist'].unique()

array([1, 0, 2])

In [56]:
SBA['NewExist'] = SBA['NewExist'].replace({2 : np.nan})
SBA['NewExist'].value_counts()

0.0    643446
1.0    252559
Name: NewExist, dtype: int64

In [57]:
SBA['NewExist'].isna().sum()

1162

## UrbanRural

In [58]:
SBA.UrbanRural.value_counts()

1    469281
0    322826
2    105060
Name: UrbanRural, dtype: int64

In [59]:
SBA.UrbanRural.isna().sum()

0

## FranchiseCode

Berdasarkan jurnal Should This Loan be Approved or Denied?”: A Large Dataset with Class Assignment Guidelines

Jika FranchiseCode = 0 atau FranchiseCode = 1 maka dia tidak ada franchise, selain itu maka ada franchise

In [60]:
SBA['FranchiseCode'].unique()

array([    1,     0, 15100, ...,  2899, 18701, 15930], dtype=int64)

In [61]:
# akan diubah nama kolom FranchiseCode menjadi IsFranchised

SBA.rename(columns={"FranchiseCode": "IsFranchised"}, inplace=True)
SBA.IsFranchised.value_counts()

1        637395
0        208040
78760      3373
68020      1921
50564      1034
          ...  
38608         1
75677         1
77030         1
19660         1
25366         1
Name: IsFranchised, Length: 2767, dtype: int64

In [62]:
SBA['IsFranchised'] = SBA['IsFranchised'].replace(1, 0)
SBA['IsFranchised'] = np.where((SBA.IsFranchised != 0 ), 1 , SBA.IsFranchised)

In [63]:
SBA['IsFranchised'].value_counts() # jadi 0 untuk tidak ada franchise dan 1 untuk ada franchise

0    845435
1     51732
Name: IsFranchised, dtype: int64

In [64]:
SBA['IsFranchised'].isna().sum()

0

## CreateJob

In [65]:
SBA['CreateJob'].unique()

array([   0,    7,   30,    5,    4,    1,   20,   10,    3,    8,   16,
         15,    6,   11,    2,   40,   55,   25,   12,   21,   50,    9,
         13,   47,   18,   17,   14,   29,   23,   35,   43,   75,   22,
         45,   27,   65,   19,   58,   48,   72,   38,   28,   24,  150,
        200,   82,   68,   41,   80,   70,   33,   97,   32,   26,   34,
         36,   31,  100,   56,   60,   90,   77,   99,   39,   44,   51,
        120,   85,   69,   95,   42,  160,   37,   57,  600,   49, 1000,
         53,   54,   46,   59,  163,  450,  456, 3000,  452,  451,  198,
         79,  454,   62,  136,   64,   52,  126,  180,   74,  303,   63,
        386,   78,   98,  455,   76,  152,  221,  110,   84,  153,  127,
       2020,  225,  453,  125,  458,  457,  174,  104,   89,  320,  154,
        300,  102,  149, 8800,  800,  130,  235, 5199,  250,  137,  500,
        121,  105,   96,  360,  255,  140,  122,  175, 1200,   66,  112,
       3500,  118,  220,  115,   73,   93,  151,  1

In [66]:
# jika > 0 maka membuat job. kalau 0 maka tidak menbuat job
# untuk yang tidak membuat job akan dibuat menjadi 1

SBA.rename(columns={"CreateJob": "IsCreatedJob"}, inplace=True)
SBA['IsCreatedJob'] = np.where((SBA.IsCreatedJob > 0 ), 1, SBA.IsCreatedJob)
SBA.IsCreatedJob.value_counts()

0    627614
1    269553
Name: IsCreatedJob, dtype: int64

## RetainedJob

In [67]:
## jika RetainedJob = 0 maka tidak memiliki karyawan tetap, maka jika Retained > 0 maka memiliki karyawan

SBA.rename(columns={"RetainedJob": "IsRetainedJob"}, inplace=True)

In [68]:
SBA['IsRetainedJob'] = np.where((SBA.IsRetainedJob > 0 ), 1,SBA.IsRetainedJob)

In [69]:
SBA.IsRetainedJob.value_counts()

1    457194
0    439973
Name: IsRetainedJob, dtype: int64

In [70]:
SBA.IsRetainedJob.isna().sum()

0

## Term

In [71]:
# Loan Term (dalam bulan)
# Berdasarkan jurnal Should This Loan be Approved or Denied?”: A Large Dataset with Class Assignment Guidelines
# 1 : Jika Term > = 240 bulan (20 tahun) perlu di back up dengan properti jika kurang
# 0 : Term < 240 bulan (20 tahun) tidak perlu di back up dengan properti

In [72]:
SBA['Term'].unique()

array([ 84,  60, 180, 240, 120,  45, 297, 162,  12, 300,  87, 114, 144,
       126,  83, 102,  80, 137,  42,  96, 167,   7,  36,  37,  26, 264,
        72,  24,   5,  54,  66, 161,  71,   4,  93, 288, 108,  10,  13,
        90,  19,  16,   3,  27, 149,  41, 246,  18,  57, 104,  82, 298,
        14,  61, 127,  58,  44,  32,  85,  48,  31, 112,  38,  73,  47,
        11, 134,  15,  79,  53,  39,   6, 255,  55, 133,  95,  35,  59,
        62,  68, 123,  46,  70, 138,  40,  52,  25,  65,  91,   1,  74,
        49, 103,  77,  86,  63,  56,  22,   0,  97,  23,  17,  69,  21,
        43,  89, 276,  92, 183,   2, 132,  34, 131,   9,  78,  99, 129,
       216,   8,  29, 289,  30, 119, 228, 168, 208,  81, 147, 125,  94,
        51, 211,  64, 111, 266,  75, 306,  28, 232, 117, 118, 309, 303,
        98, 191, 116,  76, 113, 292,  88, 166, 244, 176, 258, 203, 231,
       142,  33, 157, 165,  50, 210, 294, 301, 106,  20, 318, 229, 204,
       269, 241, 178, 115, 174, 192,  67, 100, 141, 282, 122, 15

In [73]:
SBA['RealEstate'] = SBA['Term'].apply(lambda x: 1 if x >= 240 else 0)

In [74]:
SBA['RealEstate'].value_counts()

0    745059
1    152108
Name: RealEstate, dtype: int64

In [75]:
SBA['RealEstate'].isna().sum()

0

## Kolom Baru : Recession

##### 1 : jika perusahaan aktif selama masa resesi (1 Des 2007 - 30 Jun 2009)
##### 0 : jika perusahaan tidak aktif/gagal bayar selama masa resesi (1 Des 2007 - 30 Jun 2009)

In [76]:
SBA['DaysTerm'] = SBA['Term'] * 30 # Loan Term dalam hari

In [77]:
SBA['DaysTerm'] =  SBA['Term']*30
SBA['IsActive'] = SBA['DisbursementDate'] + pd.TimedeltaIndex(SBA['DaysTerm'], unit='D')

startdate = pd.to_datetime('2007-12-1', format = "%Y-%m-%d")
enddate = pd.to_datetime('2009-06-30', format = "%Y-%m-%d")
SBA['Recession'] = SBA['IsActive'].apply(lambda x: 1 if startdate <= x <= enddate else 0)

In [78]:
SBA['Recession'].value_counts()

0    825731
1     71436
Name: Recession, dtype: int64

In [79]:
SBA['Recession'].isna().sum()

0

## NAICS

![](SBA.png)

In [80]:
SBA['NAICS'].head()

0    451120
1    722410
2    621210
3         0
4         0
Name: NAICS, dtype: int64

In [81]:
# Berdasarkan jurnal Should This Loan be Approved or Denied?”: A Large Dataset with Class Assignment Guidelines
# Sector adalah 2 angka pertama dari NAICS
SBA.rename(columns={"NAICS": "Sector"}, inplace=True)

In [82]:
r = []
for i in SBA['Sector'] :
    if i == 0 :
        r.append(0)
    else :
        a = str(i)[ : 2]
        r.append(a)

SBA['Sector'] = r
SBA['Sector'].head()

0    45
1    72
2    62
3     0
4     0
Name: Sector, dtype: object

In [83]:
def rate(i):
    sector_default = {'21':0.08, 
                      '11':0.09, '55':0.10, 
                      '62':0.10, '22':0.14, 
                      '92':0.15, '54':0.19, 
                      '42':0.19, '31':0.19,
                      '32':0.16, '33':0.14,
                      '81':0.20, '71':0.21,
                      '72':0.22, '44':0.22,
                      '45':0.23, '23':0.23,
                      '56':0.24, '61':0.24,
                      '51':0.25, '48':0.27,
                      '49':0.23, '52':0.28, 
                      '53':0.29}
    
    if i in sector_default:
        return sector_default[i]
    
    else :
        return np.nan
    

In [84]:
# Kolom baru : Sector_rate
SBA['SectorRate'] = SBA.Sector.apply(rate)
SBA['SectorRate'].head()

0    0.23
1    0.22
2    0.10
3     NaN
4     NaN
Name: SectorRate, dtype: float64

In [85]:
# ====================================================================

In [86]:
# Berdasarkan jurnal Should This Loan be Approved or Denied?”: A Large Dataset with Class Assignment Guidelines (halaman 58),
# data ini diambil dari tahun 1987 - 2014, namun karena membuat kolom baru yakni Recession yang artinya pinjamanya harus melewati massa resessi pada tahun 2007 sampai 2009
# sehingga data yang diambil hanya sampai tahun 2010 karena rerata lama pinjaman hanya selama 5 tahun atau lebih (Tanggal pencairan setelah tanggal 31-12-2010 dihapus)
SBA = SBA[SBA['DisbursementDate'] <= pd.to_datetime('2010-12-31', format = "%Y-%m-%d")]

# atau dengan kata lain dengan dimasukkannya pinjaman dengan pencairan setelah tahun 2010 akan memberikan bobot yang lebih besar 
# kepada pinjaman yang dikenakan biaya versus dibayar penuh. Lebih khusus lagi, pinjaman yang dibebankan sebelum tanggal jatuh tempo pinjaman,
# sementara pinjaman yang kemungkinan akan dibayarkan secara penuh akan melakukannya pada tanggal jatuh tempo pinjaman (yang akan melampaui akhiran set data pada tahun 2014).
# jadi hanya akan digunakan DisbursementDate sebelum tanggal 31-12-2010

In [87]:
SBA.isna().sum()

LoanNr_ChkDgt             0
Name                     12
City                     30
State                    12
Zip                       0
Bank                   1504
BankState              1511
Sector                    0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist               1127
IsCreatedJob              0
IsRetainedJob             0
IsFranchised              0
UrbanRural                0
RevLineCr            276426
LowDoc                 5384
ChgOffDate           714374
DisbursementDate          0
DisbursementGross         0
BalanceGross              0
MIS_Status                0
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
RealEstate                0
DaysTerm                  0
IsActive                  0
Recession                 0
SectorRate           201364
dtype: int64

In [88]:
# Akan di-drop kolom yang masih ada input yang kosong karena tidak ada gunanya dan sudah digantikan dengan
# kolom yang lainya untuk dilakukan EDA.
# IsActive dan DaysTerm sudah digantikan dengan Recession
# ChgOffDate karena mengintepretasikannya sama dengan MIS_Status
# Name, Bank, dan Bank State karena tidak terlalu penting untuk target (target : berikan loan atau tidak).
SBA = SBA.drop(columns=['Name', 'Bank', 'BankState', 'ChgOffDate', 'IsActive', 'DaysTerm'], axis=1)

In [89]:
SBA.isna().sum()

LoanNr_ChkDgt             0
City                     30
State                    12
Zip                       0
Sector                    0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist               1127
IsCreatedJob              0
IsRetainedJob             0
IsFranchised              0
UrbanRural                0
RevLineCr            276426
LowDoc                 5384
DisbursementDate          0
DisbursementGross         0
BalanceGross              0
MIS_Status                0
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
RealEstate                0
Recession                 0
SectorRate           201364
dtype: int64

In [90]:
#Input yang hilang pada kolom LowDoc da MIS_Status tidak bisa diinput dengan kondisi yang telah dibuat dan persentasenya sangat sedikit dibandingkan dengan data tidak kosong sehingga kita drop rownya
SBA.dropna(subset=['City', 'State','LowDoc', 'MIS_Status', 'Sector', 'SectorRate', 'RevLineCr', 'NewExist'], inplace=True)

In [91]:
SBA.isna().sum()

LoanNr_ChkDgt        0
City                 0
State                0
Zip                  0
Sector               0
ApprovalDate         0
ApprovalFY           0
Term                 0
NoEmp                0
NewExist             0
IsCreatedJob         0
IsRetainedJob        0
IsFranchised         0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementDate     0
DisbursementGross    0
BalanceGross         0
MIS_Status           0
ChgOffPrinGr         0
GrAppv               0
SBA_Appv             0
RealEstate           0
Recession            0
SectorRate           0
dtype: int64

In [92]:
# export SBA to csv
SBA.to_csv('SBA_clean.csv')